In [2]:
from google.cloud import bigquery
import pandas as pd
pd.set_option('display.width', 1000)

# Change the project name here to your project name in your account
client = bigquery.Client(project="sturdy-cable-435110-g9")

# Q1


What are the key factors influencing customer purchasing behavior?

- Analyze historical sales data to identify patterns and trends.
- <strong>Develop customer segmentation models based on purchasing behavior. (Not done)</strong>


## Google Analytics


### Exit rate by page title


In [6]:
# Exit rate by pages
query = f"""
        SELECT hits.page.pageTitle as PageTitle, COUNT(*) AS Views, SUM(totals.bounces)/COUNT(*) AS ExitRate
        FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`, UNNEST(hits) AS hits
        WHERE hits.type='PAGE'
        GROUP BY PageTitle
        ORDER BY Views DESC
        LIMIT 10
        """

rows = client.query(query)

result_df = rows.to_dataframe()

print(result_df, "\n")

/opt/anaconda3/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


                                           PageTitle   Views  ExitRate
0                                               Home  630465  0.164693
1                                Google Online Store  393684  0.510486
2                                      Shopping Cart  209266  0.020534
3  YouTube | Shop by Brand | Google Merchandise S...  157715  0.283683
4              The Google Merchandise Store - Log In  117629  0.050243
5  Men's T-Shirts | Apparel | Google Merchandise ...  101866  0.120570
6                               Store search results   78035  0.025322
7  Men's Outerwear | Apparel | Google Merchandise...   56217  0.057829
8        Backpacks | Bags | Google Merchandise Store   53449  0.017493
9                 Apparel | Google Merchandise Store   51610  0.048983 



Exit rates are the lowest when items from a specific category are being displayed


### Bounce rates by PageTitle


In [4]:
# Bounce rate by pages
query = f"""
        SELECT hits.page.pageTitle AS PageTitle, COUNT(*) AS Views, SUM(totals.bounces)/COUNT(*) AS BounceRate
        FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`, UNNEST(hits) as hits
        WHERE hits.type = 'PAGE' AND hits.hitNumber = 1
        GROUP BY PageTitle
        ORDER BY Views DESC
        LIMIT 10
        """
rows = client.query(query)

result_df = rows.to_dataframe()

print(result_df, "\n")

/opt/anaconda3/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


                                           PageTitle   Views  BounceRate
0                                Google Online Store  339352    0.592002
1                                               Home  275170    0.374627
2  YouTube | Shop by Brand | Google Merchandise S...   79314    0.563873
3  Men's T-Shirts | Apparel | Google Merchandise ...   26342    0.465758
4              The Google Merchandise Store - Log In   16321    0.361191
5                                   Page Unavailable   15965    0.404447
6                                      Shopping Cart    9428    0.454709
7  Men's Outerwear | Apparel | Google Merchandise...    7439    0.436618
8                    Bags | Google Merchandise Store    6928    0.460450
9               Drinkware | Google Merchandise Store    6834    0.557799 



# Q2


How can we improve customer retention and lifetime value?

- Calculate customer churn rates and identify at-risk customers.
- <strong>Analyze the effectiveness of current retention strategies. (Not done)</strong>


## Google Analytics


In [22]:
query = """
    SELECT *
    FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
    LIMIT 10
"""

# Execute the query
query_job = client.query(query)

# Convert the results to a DataFrame
result_df = query_job.to_dataframe()

# Get and print the column names
columns = result_df.columns.tolist()  # Convert Index to list for easier reading
print(result_df)


/opt/anaconda3/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


   visitorId  visitNumber     visitId  visitStartTime      date                                             totals                                      trafficSource                                             device                                         geoNetwork                          customDimensions                                               hits        fullVisitorId userId clientId channelGrouping  socialEngagementType
0       <NA>            1  1496349083      1496349083  20170601  {'visits': 1, 'hits': 1, 'pageviews': 1, 'time...  {'referralPath': '/analytics/web/', 'campaign'...  {'browser': 'Chrome', 'browserVersion': 'not a...  {'continent': 'Americas', 'subContinent': 'Nor...  [{'index': 4, 'value': 'North America'}]  [{'hitNumber': 1, 'time': 0, 'hour': 13, 'minu...  7098605923794129706   None     None        Referral  Not Socially Engaged
1       <NA>            2  1496385154      1496385154  20170601  {'visits': 1, 'hits': 1, 'pageviews': 1, 'time...  {'referralPa

In [5]:
query = f"""
        SELECT
            fullVisitorID,
            channelGrouping AS Channel,
            SUM(totals.totalTransactionRevenue) AS TotalRevenue,
            COUNT(fullVisitorID) AS CustomerPurchaseCount,
            trafficSource.source AS TrafficSource,
        FROM
            `bigquery-public-data.google_analytics_sample.ga_sessions_*`,
            UNNEST(hits) AS hits,
            UNNEST(hits.product) AS product
        WHERE
            _TABLE_SUFFIX BETWEEN '20160901' AND '20160930' AND
            hits.eCommerceAction.action_type = '6'
        GROUP BY fullVisitorID, Channel, TrafficSource
        ORDER BY CustomerPurchaseCount DESC
        LIMIT 10
        """

rows = client.query(query)

result_df = rows.to_dataframe()

print(result_df, "\n")

/opt/anaconda3/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


         fullVisitorID         Channel  TotalRevenue  CustomerPurchaseCount TrafficSource
0  8851822767477687842        Referral   30878760000                     96      (direct)
1   666582393118140484        Referral   20600080000                     68      (direct)
2  3033366073043100694        Referral   11512000000                     60      (direct)
3  5689475961693395998        Referral   15128780000                     54      (direct)
4  9377429831454005466  Organic Search   20728520000                     52        google
5  8436426603099391262        Referral   37197920000                     52      (direct)
6  5632276788326171571          Direct  673375500000                     42      (direct)
7  2854722262909538790  Organic Search    4431400000                     40        google
8  6374969976921419865          Direct    7172800000                     40      (direct)
9  0780206376162514125        Referral   20200000000                     40      (direct) 



The most popular channel that is used by customers with a high purchase count


In [81]:
# https://medium.com/octave-john-keells-group/a-simple-six-step-approach-to-define-customer-churn-in-retail-f401e31e57c0#:~:text=Identifying%20the%20point%20in%20which,time%20to%20proactively%20implement%20interventions.&text=Step%206%3A,is%20identified%20as%20a%20churner.
query = f"""
        -- Compute the Number of purchases made by each visitor in a day
        -- NULL transaction means that there are no transactions made
        WITH DailyPurchases AS (
            SELECT
                PARSE_DATE('%Y%m%d', _TABLE_SUFFIX) AS Date,
                fullVisitorID,
                SUM(IF(totals.transactions IS NOT NULL, totals.transactions, 0)) AS DailyPurchaseCount
            FROM
                `bigquery-public-data.google_analytics_sample.ga_sessions_*`,
                UNNEST(hits) AS hits
            GROUP BY
                Date,
                fullVisitorID
        ),
        -- Compute the 30 Day Moving Average and the 90 Day Moving Average STD
        MovingAverages AS (
            SELECT
                Date,
                fullVisitorID,
                DailyPurchaseCount,
                COALESCE(
                    SUM(DailyPurchaseCount) OVER (
                        PARTITION BY fullVisitorID
                        ORDER BY Date
                        ROWS BETWEEN 29 PRECEDING AND CURRENT ROW
                    )/30, 0
                ) AS OneMonthMA,
                COALESCE(
                    STDDEV(DailyPurchaseCount) OVER (
                        PARTITION BY fullVisitorID
                        ORDER BY Date
                        ROWS BETWEEN 89 PRECEDING AND CURRENT ROW
                    ) * (COUNT(DailyPurchaseCount) OVER (
                        PARTITION BY fullVisitorID
                        ORDER BY Date
                        ROWS BETWEEN 89 PRECEDING AND CURRENT ROW
                    ) - 1) / 89, 0
                ) AS ThreeMonthSTDMA
            FROM
                DailyPurchases
        ),
        -- Find the threshold at which the customer is considered at risk of retention
        Threshold AS (
            SELECT
                Date,
                fullVisitorID,
                DailyPurchaseCount,
                OneMonthMA,
                (OneMonthMA - ThreeMonthSTDMA) AS LowerBound
            FROM
                MovingAverages
        ),
        -- Lag the threshold by 30 Days and the visitor should not be considered at risk immediately
        LaggingThreshold AS (
            SELECT
                Date,
                fullVisitorID,
                DailyPurchaseCount,
                OneMonthMA,
                LowerBound,
                COALESCE(
                    LAG(LowerBound, 29) OVER (PARTITION BY fullVisitorID ORDER BY Date), 0
                ) AS LowerBoundOneMonthLag
            FROM
                Threshold
            ORDER BY
                Date
        ),
        -- Select only the latest date for each visitor and check for their churn status
        RankedDates AS (
            SELECT
                Date,
                fullVisitorID,
                DailyPurchaseCount,
                OneMonthMA,
                LowerBound,
                LowerBoundOneMonthLag,
                ROW_NUMBER() OVER (
                    PARTITION BY fullVisitorID
                    ORDER BY Date DESC
                ) AS rn
            FROM LaggingThreshold
        )

        SELECT Date, fullVisitorID, DailyPurchaseCount, OneMonthMA, LowerBound, LowerBoundOneMonthLag
        FROM RankedDates
        ORDER BY Date
        """

rows = client.query(query)

result_df = rows.to_dataframe()

grouped_sum = result_df.groupby('fullVisitorID')[
    'DailyPurchaseCount'].sum().reset_index()

grouped_sum = grouped_sum.rename(
    columns={'DailyPurchaseCount': 'HistoricPurchases'})

df = result_df.merge(grouped_sum, on='fullVisitorID',
                     how='left').query("HistoricPurchases > 0")

df['Date'] = pd.to_datetime(df['Date'])

latest_idx = df.groupby('fullVisitorID')['Date'].idxmax()

result_df = df.loc[latest_idx].sort_values(by="Date")

print(result_df, "\n")

             Date        fullVisitorID  DailyPurchaseCount  OneMonthMA  LowerBound  LowerBoundOneMonthLag  HistoricPurchases
408    2016-08-01  5760753352577829144                  79    2.633333    2.633333                    0.0                 79
717    2016-08-01  6569605994631186947                  36    1.200000    1.200000                    0.0                 36
80     2016-08-01  2125540555068339394                  25    0.833333    0.833333                    0.0                 25
432    2016-08-01  7589137567725941774                  51    1.700000    1.700000                    0.0                 51
756    2016-08-01  5563168194966233133                  21    0.700000    0.700000                    0.0                 21
...           ...                  ...                 ...         ...         ...                    ...                ...
834084 2017-08-01   024507252193437459                  16    0.533333    0.406213                    0.0                 16


/var/folders/lj/tfjxb4wd4kx1kwshmxnt57840000gn/T/ipykernel_33309/253609387.py:103: RuntimeWarning: Engine has switched to 'python' because numexpr does not support extension array dtypes. Please set your engine to python manually.
  how='left').query("HistoricPurchases > 0")


In [82]:
# Those that have not made a purchase within the last 3 months or have a monthly moving average less than the lower bound have a retention risk
max_date = result_df['Date'].max()
three_months_ago = max_date - pd.DateOffset(months=3)
RetentionRiskCount = len(set(result_df.query(
    "Date <= @three_months_ago | LowerBoundOneMonthLag >= OneMonthMA").fullVisitorID))

# Those that have made a purchase within the last 3 months or have a monthly moving average more than the lower bound do not have a retention risk
RetentionCount = len(set(result_df.query(
    "@three_months_ago <= Date <= @max_date & LowerBoundOneMonthLag < OneMonthMA").fullVisitorID))


result_df.query(
    "@three_months_ago <= Date <= @max_date & LowerBoundOneMonthLag < OneMonthMA")

,Date,fullVisitorID,DailyPurchaseCount,OneMonthMA,LowerBound,LowerBoundOneMonthLag,HistoricPurchases
650611,2017-05-01,536428604379922267,22,0.733333,0.733333,0.0,22
651870,2017-05-01,1749853719127085935,13,0.433333,0.433333,0.0,13
650822,2017-05-01,9740050408987139130,19,0.633333,0.633333,0.0,19
650153,2017-05-01,9565722193439879038,11,0.366667,0.279271,0.0,11
650942,2017-05-01,4501448098783068461,22,2.466667,1.636650,0.0,74
...,...,...,...,...,...,...,...
834084,2017-08-01,024507252193437459,16,0.533333,0.406213,0.0,16
835321,2017-08-01,5199370466032130686,0,0.866667,0.428464,0.0,26
834618,2017-08-01,3614707430894059857,28,0.933333,0.933333,0.0,28
833186,2017-08-01,9591202457292182670,20,0.666667,0.157052,0.0,20


These are the at risk customers as the lagged lower bound is greater or equal to their one month moving average on purchase count


In [9]:
query = f"""
        -- Compute the Number of purchases made by each visitor in a day
        -- NULL transaction means that there are no transactions made
        WITH DailyPurchases AS (
            SELECT
                PARSE_DATE('%Y%m%d', _TABLE_SUFFIX) AS Date,
                fullVisitorID,
                SUM(IF(totals.transactions IS NOT NULL, totals.transactions, 0)) AS DailyPurchaseCount
            FROM
                `bigquery-public-data.google_analytics_sample.ga_sessions_*`,
                UNNEST(hits) AS hits
            GROUP BY
                Date,
                fullVisitorID
        ),
        -- Compute the 30 Day Moving Average and the 90 Day Moving Average STD
        MovingAverages AS (
            SELECT
                Date,
                fullVisitorID,
                DailyPurchaseCount,
                COALESCE(
                    SUM(DailyPurchaseCount) OVER (
                        PARTITION BY fullVisitorID
                        ORDER BY Date
                        ROWS BETWEEN 29 PRECEDING AND CURRENT ROW
                    )/30, 0
                ) AS OneMonthMA,
                COALESCE(
                    STDDEV(DailyPurchaseCount) OVER (
                        PARTITION BY fullVisitorID
                        ORDER BY Date
                        ROWS BETWEEN 89 PRECEDING AND CURRENT ROW
                    ) * (COUNT(DailyPurchaseCount) OVER (
                        PARTITION BY fullVisitorID
                        ORDER BY Date
                        ROWS BETWEEN 89 PRECEDING AND CURRENT ROW
                    ) - 1) / 89, 0
                ) AS ThreeMonthSTDMA
            FROM
                DailyPurchases
        ),
        -- Find the threshold at which the customer is considered at risk of retention
        Threshold AS (
            SELECT
                Date,
                fullVisitorID,
                DailyPurchaseCount,
                OneMonthMA,
                (OneMonthMA - ThreeMonthSTDMA) AS LowerBound
            FROM
                MovingAverages
        ),
        -- Lag the threshold by 30 Days and the visitor should not be considered at risk immediately
        LaggingThreshold AS (
            SELECT
                Date,
                fullVisitorID,
                DailyPurchaseCount,
                OneMonthMA,
                LowerBound,
                COALESCE(
                    LAG(LowerBound, 29) OVER (PARTITION BY fullVisitorID ORDER BY Date), 0
                ) AS LowerBoundOneMonthLag
            FROM
                Threshold
            ORDER BY
                Date
        ),
        -- Find the channels used by those with a retention risk: Less than threshold or have not purchased within the last 3 months
        RetentionRisk AS (
            SELECT
                channelGrouping AS Channel,
                SUM(IF(totals.transactions IS NOT NULL, totals.transactions, 0)) AS Transactions
            FROM
                `bigquery-public-data.google_analytics_sample.ga_sessions_*` df
            RIGHT JOIN
                LaggingThreshold lt
            ON
                df.fullVisitorID = lt.fullVisitorID
            WHERE
                lt.LowerBoundOneMonthLag >= lt.OneMonthMA AND lt.Date < (SELECT MAX(Date) FROM LaggingThreshold) - INTERVAL 3 MONTH
            GROUP BY
                Channel
        ),
        TotalTransactions AS (
            SELECT
                SUM(Transactions) AS Total
            FROM
                RetentionRisk
        )

        SELECT
            Channel,
            Transactions,
            (Transactions / TotalTransactions.Total) * 100 AS Transactions_Pct
        FROM
            RetentionRisk,
            TotalTransactions
        ORDER BY
            Transactions_Pct DESC
        """

rows = client.query(query)

result_df = rows.to_dataframe()

print(result_df, "\n")

          Channel  Transactions  Transactions_Pct
0        Referral          4419         43.154297
1  Organic Search          2360         23.046875
2         Display          1668         16.289062
3          Direct          1455         14.208984
4     Paid Search           239          2.333984
5          Social            96          0.937500
6      Affiliates             3          0.029297
7         (Other)             0          0.000000 



Overview of channels used by those who do not have a retention risk


In [8]:
query = f"""
        -- Compute the Number of purchases made by each visitor in a day
        -- NULL transaction means that there are no transactions made
        WITH DailyPurchases AS (
            SELECT
                PARSE_DATE('%Y%m%d', _TABLE_SUFFIX) AS Date,
                fullVisitorID,
                SUM(IF(totals.transactions IS NOT NULL, totals.transactions, 0)) AS DailyPurchaseCount
            FROM
                `bigquery-public-data.google_analytics_sample.ga_sessions_*`,
                UNNEST(hits) AS hits
            GROUP BY
                Date,
                fullVisitorID
        ),
        -- Compute the 30 Day Moving Average and the 90 Day Moving Average STD
        MovingAverages AS (
            SELECT
                Date,
                fullVisitorID,
                DailyPurchaseCount,
                COALESCE(
                    SUM(DailyPurchaseCount) OVER (
                        PARTITION BY fullVisitorID
                        ORDER BY Date
                        ROWS BETWEEN 29 PRECEDING AND CURRENT ROW
                    )/30, 0
                ) AS OneMonthMA,
                COALESCE(
                    STDDEV(DailyPurchaseCount) OVER (
                        PARTITION BY fullVisitorID
                        ORDER BY Date
                        ROWS BETWEEN 89 PRECEDING AND CURRENT ROW
                    ) * (COUNT(DailyPurchaseCount) OVER (
                        PARTITION BY fullVisitorID
                        ORDER BY Date
                        ROWS BETWEEN 89 PRECEDING AND CURRENT ROW
                    ) - 1) / 89, 0
                ) AS ThreeMonthSTDMA
            FROM
                DailyPurchases
        ),
        -- Find the threshold at which the customer is considered at risk of retention
        Threshold AS (
            SELECT
                Date,
                fullVisitorID,
                DailyPurchaseCount,
                OneMonthMA,
                (OneMonthMA - ThreeMonthSTDMA) AS LowerBound
            FROM
                MovingAverages
        ),
        -- Lag the threshold by 30 Days and the visitor should not be considered at risk immediately
        LaggingThreshold AS (
            SELECT
                Date,
                fullVisitorID,
                DailyPurchaseCount,
                OneMonthMA,
                LowerBound,
                COALESCE(
                    LAG(LowerBound, 29) OVER (PARTITION BY fullVisitorID ORDER BY Date), 0
                ) AS LowerBoundOneMonthLag
            FROM
                Threshold
            ORDER BY
                Date
        ),
        -- Find the channels used by those without a retention risk: More than threshold or have purchased within the last 3 months
        Retention AS (
            SELECT
                channelGrouping AS Channel,
                SUM(IF(totals.transactions IS NOT NULL, totals.transactions, 0)) AS Transactions
            FROM
                `bigquery-public-data.google_analytics_sample.ga_sessions_*` df
            RIGHT JOIN
                LaggingThreshold lt
            ON
                df.fullVisitorID = lt.fullVisitorID
            WHERE
                lt.LowerBoundOneMonthLag < lt.OneMonthMA AND lt.Date >= (SELECT MAX(Date) FROM LaggingThreshold) - INTERVAL 3 MONTH
            GROUP BY
                Channel
        ),
        TotalTransactions AS (
            SELECT
                SUM(Transactions) AS Total
            FROM
                Retention
        )

        SELECT
            Channel,
            Transactions,
            (Transactions / TotalTransactions.Total) * 100 AS Transactions_Pct
        FROM
            Retention,
            TotalTransactions
        ORDER BY
            Transactions_Pct DESC
        """

rows = client.query(query)

result_df = rows.to_dataframe()

print(result_df, "\n")

          Channel  Transactions  Transactions_Pct
0        Referral          3695         39.812520
1          Direct          2193         23.628919
2  Organic Search          2133         22.982437
3         Display           884          9.524836
4     Paid Search           307          3.307833
5          Social            63          0.678806
6      Affiliates             6          0.064648
7         (Other)             0          0.000000 



Overview of channels used by those who do not have a retention risk


In [83]:
ChurnRate = (RetentionRiskCount /
             (RetentionRiskCount + RetentionCount)) * 100

print("The churn rate for google online store is :", ChurnRate, "% \n")

The churn rate for google online store is : 69.27042030134814 % 



In [27]:
query = """
    SELECT 
        fullVisitorId,
        SUM(totals.pageviews) AS total_pageviews,
        AVG(totals.timeOnSite) AS avg_time_on_site
        FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
    WHERE 
        _TABLE_SUFFIX BETWEEN '20160901' AND '20160930'
    GROUP BY fullVisitorId
    ORDER BY total_pageviews DESC

    LIMIT 100;
    """

df = client.query(query).to_dataframe()

print(df)


/opt/anaconda3/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


          fullVisitorId  total_pageviews  avg_time_on_site
0   2194592743396253647              647        796.298507
1   0824839726118485274              576       2057.542857
2   1856749147915772585              556        947.157895
3   8851822767477687842              393        992.875000
4   4086679804074527652              381       1266.333333
..                  ...              ...               ...
95   183286922797817588              126        972.500000
96  9538778820434741432              125       2066.571429
97  4174946983461755352              125       1754.000000
98  0003450834640354121              124        853.500000
99  3447003791318310120              123       2444.000000

[100 rows x 3 columns]


In [35]:
query = """
SELECT 
    visitId,
    fullVisitorId,
    date,
    totals.visits,
    totals.pageviews,
    totals.timeOnSite,
    totals.transactions,
    totals.transactionRevenue,
    trafficSource.source,
    geoNetwork.country,
    device.deviceCategory,
    COUNT(DISTINCT visitId) AS unique_visits,
    AVG(totals.timeOnSite) AS avg_time_on_site,
    SUM(totals.transactions) AS total_transactions,
    SUM(totals.transactionRevenue) AS total_revenue
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE _TABLE_SUFFIX BETWEEN '20160901' AND '20160930'
GROUP BY 
    visitId, fullVisitorId, date, totals.visits, totals.pageviews, totals.timeOnSite, totals.transactions, totals.transactionRevenue, trafficSource.source, geoNetwork.country, device.deviceCategory
ORDER BY 
    total_revenue DESC
LIMIT 1000

"""

rows = client.query(query)

result_df = rows.to_dataframe()

print(result_df)

/opt/anaconda3/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


        visitId        fullVisitorId      date  visits  pageviews  timeOnSite  transactions  transactionRevenue       source        country deviceCategory  unique_visits  avg_time_on_site  total_transactions  total_revenue
0    1474035643  5632276788326171571  20160916       1         89        1486             1         16023750000     (direct)  United States        desktop              1            1486.0                   1    16023750000
1    1474965281  4067986469497332117  20160927       1         26        3883             1          3388800000     (direct)          Japan        desktop              1            3883.0                   1     3388800000
2    1474931750  1044832976156649069  20160926       1         31        1257             1          2399700000     (direct)         Canada        desktop              1            1257.0                   1     2399700000
3    1473950759  3979618861854336423  20160915       1        202        6596             1          1890180

In [36]:
query = """
WITH customer_segments AS (
    SELECT 
        fullVisitorId,
        COUNT(DISTINCT visitId) AS visit_count,
        AVG(totals.timeOnSite) AS avg_time_on_site,
        SUM(totals.transactions) AS total_transactions,
        SUM(totals.transactionRevenue) AS total_revenue,
        geoNetwork.country,
        device.deviceCategory
    FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
    WHERE _TABLE_SUFFIX BETWEEN '20160901' AND '20160930'
    GROUP BY 
        fullVisitorId, geoNetwork.country, device.deviceCategory
)
SELECT 
    country,
    deviceCategory,
    AVG(visit_count) AS avg_visits,
    AVG(total_transactions) AS avg_transactions,
    AVG(total_revenue) AS avg_revenue
FROM 
    customer_segments
GROUP BY 
    country, deviceCategory
ORDER BY 
    avg_revenue DESC
"""

rows = client.query(query)

result_df = rows.to_dataframe()

print(result_df)

/opt/anaconda3/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


           country deviceCategory  avg_visits  avg_transactions   avg_revenue
0            Japan        desktop    1.133264               1.0  3.388800e+09
1           Canada        desktop    1.401084               1.0  4.599044e+08
2           Taiwan        desktop    1.218989               2.0  2.583300e+08
3        Hong Kong        desktop    1.243902               1.0  2.396650e+08
4      Puerto Rico        desktop    1.120000               1.0  2.249700e+08
..             ...            ...         ...               ...           ...
399          Malta         mobile    1.000000               NaN           NaN
400         Kuwait         tablet    1.000000               NaN           NaN
401        Réunion         tablet    1.000000               NaN           NaN
402      Hong Kong         tablet    1.000000               NaN           NaN
403  French Guiana         mobile    1.000000               NaN           NaN

[404 rows x 5 columns]


In [45]:
query = """
SELECT
    CASE WHEN totals.newVisits = 1 THEN 'New' ELSE 'Returning' END AS visitor_type,
    COUNT(DISTINCT fullVisitorId) AS visitor_count,
    SUM(totals.pageviews) AS total_pageviews,
    AVG(totals.timeOnSite) AS avg_time_on_site,
    AVG(totals.bounces) AS avg_bounces
FROM
    `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE _TABLE_SUFFIX BETWEEN '20160901' AND '20160930'
GROUP BY
    visitor_type
"""

query_job = client.query(query)

result_df = query_job.to_dataframe()


print(result_df)

/opt/anaconda3/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


  visitor_type  visitor_count  total_pageviews  avg_time_on_site  avg_bounces
0    Returning           8921           105640        413.895152          1.0
1          New          55555           221723        235.985886          1.0


In [48]:
query = """
    WITH UserVisits AS (
        SELECT
            fullVisitorId,
            visitNumber
        FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
        WHERE _TABLE_SUFFIX BETWEEN '20160901' AND '20160930'
        GROUP BY fullVisitorId, visitNumber
    )

    SELECT
        CASE
            WHEN UserVisits.visitNumber = 1 THEN 'New User'
            ELSE 'Returning User'
        END AS UserType,
        COUNT(*) AS Sessions,
        AVG(sessions.totals.timeOnSite) AS AvgSessionDuration,
        SUM(sessions.totals.bounces) / COUNT(*) AS BounceRate
    FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` AS sessions
    JOIN UserVisits
    ON sessions.fullVisitorId = UserVisits.fullVisitorId
    WHERE _TABLE_SUFFIX BETWEEN '20160901' AND '20160930'
    GROUP BY UserType
    ORDER BY Sessions DESC
"""

row = client.query(query)

result_df = row.to_dataframe()

print(result_df)

/opt/anaconda3/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


         UserType  Sessions  AvgSessionDuration  BounceRate
0  Returning User     79806          585.249634    0.398980
1        New User     64252          265.122362    0.503969


In [42]:
query = """
    SELECT
        device.deviceCategory,
        AVG(totals.timeOnSite) AS avg_time_on_site,
        AVG(totals.pageviews) AS avg_pageviews,
        AVG(totals.bounces) AS avg_bounce_rate
    FROM
        `bigquery-public-data.google_analytics_sample.ga_sessions_*`
    WHERE 
        _TABLE_SUFFIX BETWEEN '20160901' AND '20160930'
    GROUP BY
        device.deviceCategory
    """

rows = client.query(query)

result_df = rows.to_dataframe()

print(result_df)

/opt/anaconda3/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


  deviceCategory  avg_time_on_site  avg_pageviews  avg_bounce_rate
0        desktop        283.196537       4.632363              1.0
1         mobile        268.329531       4.369049              1.0
2         tablet        295.453237       5.385432              1.0


In [46]:
query = """
    SELECT
        fullVisitorId,
        SUM(totals.pageviews) AS total_pageviews,
        SUM(totals.hits) AS total_hits,
        AVG(totals.timeOnSite) AS avg_time_on_site,
        SUM(totals.bounces) AS total_bounces
    FROM
        `bigquery-public-data.google_analytics_sample.ga_sessions_*`
    WHERE 
        _TABLE_SUFFIX BETWEEN '20160901' AND '20160930'
    GROUP BY
        fullVisitorId
    ORDER BY
        total_pageviews DESC
    """

rows = client.query(query)

result_df = rows.to_dataframe()

print(result_df)

/opt/anaconda3/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


             fullVisitorId  total_pageviews  total_hits  avg_time_on_site  total_bounces
0      2194592743396253647              647         757        796.298507             19
1      0824839726118485274              576         717       2057.542857             10
2      1856749147915772585              556         666        947.157895              7
3      8851822767477687842              393         557        992.875000              2
4      4086679804074527652              381         567       1266.333333           <NA>
...                    ...              ...         ...               ...            ...
59116   246886865255684914                1           1               NaN              1
59117  0670569165766150000                1           1               NaN              1
59118  2154708034844551478                1           1               NaN              1
59119  1335370086170802517                1           1               NaN              1
59120  62721146851801

In [49]:
query = """
    SELECT
        fullVisitorId,
        COUNT(totals.transactions) AS total_transactions,
        SUM(totals.transactionRevenue) AS total_revenue,
        AVG(totals.transactionRevenue) AS avg_revenue_per_transaction
    FROM
        `bigquery-public-data.google_analytics_sample.ga_sessions_*`
    WHERE
        totals.transactions IS NOT NULL
    GROUP BY
        fullVisitorId
    ORDER BY
        total_revenue DESC
    """

rows = client.query(query)

result_df = rows.to_dataframe()

print(result_df)

/opt/anaconda3/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


             fullVisitorId  total_transactions  total_revenue  avg_revenue_per_transaction
0      1957458976293878100                  16    77113430000                 4.819589e+09
1      5632276788326171571                   1    16023750000                 1.602375e+10
2      9417857471295131045                   5    15170120000                 3.034024e+09
3      4471415710206918415                   4    11211100000                 2.802775e+09
4      4984366501121503466                  16     9513900000                 5.946188e+08
...                    ...                 ...            ...                          ...
10017  0628708827036512143                   1           <NA>                          NaN
10018  4330494225147030149                   1           <NA>                          NaN
10019  4019173914403244131                   1           <NA>                          NaN
10020  9137922581159198144                   1           <NA>                          NaN

In [62]:
query_social_engagement = """
    SELECT
        channelGrouping,
        SUM(totals.transactions) AS total_transactions,
        SUM(totals.transactionRevenue) AS total_revenue,
        COUNT(DISTINCT fullVisitorId) AS total_visitors
    FROM
        `bigquery-public-data.google_analytics_sample.ga_sessions_*`,
        UNNEST(hits) AS hit
    GROUP BY
        channelGrouping
    """

df_social_engagement = client.query(query_social_engagement).to_dataframe()
print(df_social_engagement.head())

/opt/anaconda3/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


  channelGrouping  total_transactions   total_revenue  total_visitors
0  Organic Search              149447  18235868990000          311607
1         Display                5983   5120535430000            4103
2        Referral              210812  28666307920000           65661
3      Affiliates                 334     27260690000           13400
4          Direct              102671  23535451680000          109830


# Q3


What are the most effective marketing channels and campaigns? - Evaluate the ROI of different marketing channels.

- Analyze the impact of various promotional campaigns on sales.


## Google Analytics


In [12]:
query = f"""
        SELECT
            channelGrouping AS Channel,
            trafficSource.source AS TrafficSource,
            CASE
                WHEN hits.eCommerceAction.action_type = '1' THEN 'Click product lists'
                WHEN hits.eCommerceAction.action_type = '2' THEN 'View product details'
                WHEN hits.eCommerceAction.action_type = '3' THEN 'Add to cart'
                WHEN hits.eCommerceAction.action_type = '4' THEN 'Remove from cart'
                WHEN hits.eCommerceAction.action_type = '5' THEN 'Check out'
                WHEN hits.eCommerceAction.action_type = '6' THEN 'Purchase'
                WHEN hits.eCommerceAction.action_type = '7' THEN 'Refund'
                WHEN hits.eCommerceAction.action_type = '0' THEN 'Unknown'
            END AS Action,
            COUNT(fullVisitorID) AS UserCount
        FROM
            `bigquery-public-data.google_analytics_sample.ga_sessions_*`,
            UNNEST(hits) AS hits,
            UNNEST(hits.product) AS product
        WHERE
            hits.eCommerceAction.action_type != '0'
        GROUP BY Action, TrafficSource, Channel
        ORDER BY UserCount DESC
        LIMIT 10
        """

rows = client.query(query)

result_df = rows.to_dataframe()

print(result_df, "\n")

          Channel TrafficSource                Action  UserCount
0  Organic Search        google   Click product lists     130574
1  Organic Search        google  View product details     105817
2        Referral      (direct)   Click product lists      95294
3        Referral      (direct)             Check out      95200
4  Organic Search      (direct)   Click product lists      80034
5          Direct      (direct)   Click product lists      78922
6        Referral      (direct)  View product details      77032
7  Organic Search      (direct)  View product details      65341
8          Direct      (direct)  View product details      63953
9  Organic Search        google             Check out      54376 



Using the dataframe above, we can find which platform and method is the most effective in influencing sales


In [13]:
query = f"""
        SELECT
            channelGrouping AS Channel,
            SUM(totals.totalTransactionRevenue) AS TotalRevenue,
            COUNT(fullVisitorID) AS UserCount,
            SUM(totals.totalTransactionRevenue) / COUNT(fullVisitorID) AS AVGRevenuePerUser,
            trafficSource.source AS TrafficSource,
            CASE
                WHEN hits.eCommerceAction.action_type = '1' THEN 'Click product lists'
                WHEN hits.eCommerceAction.action_type = '2' THEN 'View product details'
                WHEN hits.eCommerceAction.action_type = '3' THEN 'Add to cart'
                WHEN hits.eCommerceAction.action_type = '4' THEN 'Remove from cart'
                WHEN hits.eCommerceAction.action_type = '5' THEN 'Check out'
                WHEN hits.eCommerceAction.action_type = '6' THEN 'Purchase'
                WHEN hits.eCommerceAction.action_type = '7' THEN 'Refund'
                WHEN hits.eCommerceAction.action_type = '0' THEN 'Unknown'
            END AS Action
        FROM
            `bigquery-public-data.google_analytics_sample.ga_sessions_*`,
            UNNEST(hits) AS hits,
            UNNEST(hits.product) AS product
        WHERE
            hits.eCommerceAction.action_type = '6'
        GROUP BY Action, TrafficSource, Channel
        ORDER BY UserCount DESC
        LIMIT 20
        """

rows = client.query(query)

result_df = rows.to_dataframe()

print(result_df, "\n")

           Channel   TotalRevenue  UserCount  AVGRevenuePerUser            TrafficSource    Action
0         Referral  6861093620000      32354       2.120632e+08                 (direct)  Purchase
1           Direct  9231074220000      14684       6.286485e+08                 (direct)  Purchase
2   Organic Search  3403071550000      13419       2.536010e+08                   google  Purchase
3   Organic Search  1709765760000       7592       2.252062e+08                 (direct)  Purchase
4      Paid Search   361601340000       1628       2.221138e+08                   google  Purchase
5      Paid Search   207373340000       1296       1.600103e+08                 (direct)  Purchase
6          Display  4605551900000       1010       4.559952e+09                      dfa  Purchase
7         Referral   374165040000        578       6.473444e+08          mail.google.com  Purchase
8         Referral    84159360000        356       2.364027e+08            dealspotr.com  Purchase
9         

Referral searches is currently the most popular method when purchasing an item

Direct searches generates a greater revenue


In [144]:
# For each page title, find the number of customers and the way people are able to search for their products -> Which is the most effective
query = f"""
        SELECT
            IF(LENGTH(hits.page.pageTitle) <= 20, hits.page.pageTitle, CONCAT(SUBSTR(hits.page.pageTitle, 1, 20), '...')) AS PageTitle,
            channelGrouping AS Channel,
            COUNT(fullVisitorID) AS VisitorCount,
            SUM(IF(totals.transactions IS NULL, 0, totals.transactions)) AS TransactionCount
        FROM
            `bigquery-public-data.google_analytics_sample.ga_sessions_*`, UNNEST(hits) AS hits
        WHERE hits.page.pageTitle NOT LIKE '%Checkout%'
            AND hits.page.pageTitle NOT LIKE '%Home%'
            AND hits.page.pageTitle NOT LIKE '%Cart%'
            AND hits.page.pageTitle NOT LIKE '%Google%'
            AND hits.page.pageTitle NOT LIKE '%Payment%'
            AND hits.page.pageTitle NOT LIKE '%results%'
        GROUP BY PageTitle, Channel
        ORDER BY VisitorCount DESC
        LIMIT 20
        """

rows = client.query(query)

result_df = rows.to_dataframe()

print(result_df, "\n")

           PageTitle         Channel  VisitorCount  TransactionCount
0     Men's-T-Shirts  Organic Search         31922              2698
1             Office  Organic Search         22441              2107
2   Page Unavailable        Referral         21349               499
3               Bags  Organic Search         19002              1288
4        Electronics  Organic Search         18657              2486
5          Drinkware  Organic Search         17328              1861
6     Men's-T-Shirts        Referral         14478              3573
7            Apparel  Organic Search         12960               899
8          Lifestyle  Organic Search         12844              1428
9               Bags        Referral         11839              1474
10            Office        Referral         11747              2691
11           YouTube  Organic Search         11369               472
12       Electronics        Referral         11298              1629
13         Drinkware        Referr

Referral and organic searches are clearly the most popular traffic channels and a higher traffic generally equates to a higher transaction count
